<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/cookbooks/upstage/Solar-Full-Stack LLM-101/05_3_OracleDB.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Retrieval Augmented Generation (RAG) Baseline
## Overview  
In this time, we will check the baseline code.
The goal of this project is to provide students with hands-on experience in handling and enhancing Large Language Models (LLMs) provided by [**Upstage**](https://www.upstage.ai) (Solar).

You can use any engineering method for improving benchmark performance excluding direct training (Fine-tuning).

*Collecting data directly related to the test set is considered cheating e.g., using MMLU-pro dataset or EWHA.pdf for KB*

# Baseline

In [ ]:
#!pip3 install -qU python-dotenv PyPDF2 langchain langchain-community langchain-core langchain-text-splitters langchain_upstage oracledb python-dotenv

In [ ]:
# Additional Contents : https://wikidocs.net/253106 

In [2]:
# set parameters

file = open("info/api.txt", "r")
api_key = file.read()
file.close()
file = open("info/datapath.txt", "r")
data_path = file.read()
file.close()

In [3]:
from langchain_upstage import UpstageEmbeddings

# 쿼리 전용 임베딩 모델
query_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-query")

# 문장 전용 임베딩 모델
passage_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-passage")

## 1. build DB

In [4]:
from langchain_upstage import UpstageLayoutAnalysisLoader
import os
import numpy as np


UPSTAGE_API_KEY = api_key

# .npy 파일 로드 (타입==넘파이)
ewhaDB = np.load(data_path+f'embedding/full_ewha500.npy')

ewhaDB_embed = np.load(data_path+f'embedding/full_ewha500_embed.npy')
ewhaDB_embed = ewhaDB_embed.tolist()

## 2. Text Split

## 3. query-context matching

In [5]:
# read samples.csv file
import pandas as pd

def read_data(data_path):
    data = pd.read_csv(data_path)
    prompts = data['prompts']
    answers = data['answers']
    # returns two lists: prompts and answers
    return prompts, answers

In [6]:
prompts, answers = read_data(os.path.join(data_path, 'test_own_ewha.csv'))

In [7]:
testdata = pd.read_csv(data_path+'test_own_ewha.csv')
testdata

,prompts,answers
0,QUESTION1) 영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한...,(D)
1,QUESTION2) 각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n...,(E)
2,QUESTION3) 부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속...,(B)
3,QUESTION4) 학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?...,(B)
4,QUESTION5) 학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하...,(C)
5,QUESTION6) 학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇...,(D)
6,QUESTION7) 학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인...,(C)
7,QUESTION8) 학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한...,(D)
8,QUESTION9) 학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 ...,(C)
9,QUESTION10) 학칙 제50조(졸업)에서 복수전공 이수자가 요구되는 추가 조건...,(B)


In [8]:
nowtest = pd.DataFrame(columns=['index', 'embed_ques', 'question', 'prompts', 'answers', 'top1', 'top2', 'top3'])

for index, row in testdata.iterrows():
    #if index % 100 != 0 : continue # 일단 실험할 땐 100개 단위로 끊어서 가져옴
    q = row.prompts
    a = row.answers
    question = q.partition('(A)')[0]
    question = question.partition(') ')[2]
    q = q.partition(') ')[2]
    try : 
        embedded_query = query_embeddings.embed_query(question) # 질문만 받아와서 embedding 하기
        nowtest.loc[len(nowtest)] = {'index':index, 'embed_ques' : embedded_query, 'question' : question, 'prompts' : q, 'answers' : a, 'top1': None, 'top2': None, 'top3': None}

    except :
        print(f'pass: {index}')
        continue


In [9]:
nowtest

,index,embed_ques,question,prompts,answers,top1,top2,top3
0,0,"[-0.005329132080078125, 0.0017337799072265625,...",영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학...,영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학...,(D),None,None,None
1,1,"[-0.00460052490234375, -0.0174713134765625, -0...",각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n,각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n(A) 이학사\n(B...,(E),None,None,None
2,2,"[0.007404327392578125, -0.0216522216796875, 0....",부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속하는가?\n,부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속하는가?\n(A) 현...,(B),None,None,None
3,3,"[-0.002162933349609375, -0.039215087890625, -0...",학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?\n,학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?\n(A) 시험 직후...,(B),None,None,None
4,4,"[0.0089111328125, -0.0259552001953125, -0.0064...",학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 무엇입...,학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 무엇입...,(C),None,None,None
5,5,"[0.0016117095947265625, -0.019683837890625, -0...",학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇입니까?\n,학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇입니까?\n(A) 서...,(D),None,None,None
6,6,"[-0.018157958984375, -0.0157318115234375, -0.0...",학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인정되지 않는 경우는?\n,학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인정되지 않는 경우는?...,(C),None,None,None
7,7,"[0.0029163360595703125, -0.018707275390625, -0...",학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한이 다른 이유로 올바...,학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한이 다른 이유로 올바...,(D),None,None,None
8,8,"[-5.4895877838134766e-05, -0.01477813720703125...",학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 초과하는 경우가 아닌...,학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 초과하는 경우가 아닌...,(C),None,None,None
9,9,"[-0.002269744873046875, 0.0017795562744140625,...",학칙 제50조(졸업)에서 복수전공 이수자가 요구되는 추가 조건은 무엇입니까?\n,학칙 제50조(졸업)에서 복수전공 이수자가 요구되는 추가 조건은 무엇입니까?\n(A...,(B),None,None,None


## 4. Prompt engineering

In [10]:
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    You are an assistant with expertise in Ewha University policies and history. Use the provided context to answer accurately.

    Context: {context}

    Question: {question}

    Final Answer:

    '''

)
chain = prompt_template | llm




In [92]:
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    
    [Question]에 대한 정답을 출력해. 주어진 [Context]에서 관련된 정보를 찾아.
    
    정확하게 정답만 답변해.
    그리고 답변이 정확한지 한 번 더 확인해
    ---
    
    [Question] {question}
    [Context] {context}
    [Answer]
        
    '''

)
ko_chain = prompt_template | llm


# top1 = []
# for idx, row in nowtest.iterrows() : ############### 일단 지금은 100개 단위로 띄엄띄엄 test 중
#     response = ko_chain.invoke({"question": row.question}) # 선지 전까지 받아오기
#     top1.append(response.content)
#     nowtest.loc[idx, 'topic_b'] = response.content

# top1

In [ ]:
import numpy as np

chunk = []
for index in range(0,len(splits)) : # context 받아오기
    chunk.append(splits[index].page_content)

responses = []
for idx, row in nowtest.iterrows() : # 질문 받아오기 
    
    embedded_query = query_embeddings.embed_query(row.prompt.split('\n')[0]) # 선지 빼고 가져오기


    # 유사도 기준 내림차순 정렬
    sorted_idx = (np.array(embedded_query) @ np.array(embedded_documents).T).argsort()[::-1]

    # QA
    print("*****************************************")
    
    print(f"now prompt: {row.prompt.split('\n')[0]}")
    print(sorted_idx[0])
    print(chunk[sorted_idx[0]])

    print("*****************************************")

    response = chain.invoke({"question": prompt, "context": sorted_idx[0]})
    responses.append(response.content)


NameError: name 'splits' is not defined

In [ ]:
for i in responses:
    print(i)
    print('-')


## Check Accuracy

In [ ]:
# funcion to extract an answer from response

import re

def extract_answer(response):
    """
    extracts the answer from the response using a regular expression.
    expected format: "[ANSWER]: (A) convolutional networks"

    if there are any answers formatted like the format, it returns None.
    """
    pattern = r"\[ANSWER\]:\s*\((A|B|C|D|E)\)"  # Regular expression to capture the answer letter and text
    match = re.search(pattern, response)

    if match:
        return match.group(1) # Extract the letter inside parentheses (e.g., A)
    else:
        return extract_again(response)

def extract_again(response):
    pattern = r"\b[A-J]\b(?!.*\b[A-J]\b)"
    match = re.search(pattern, response)
    if match:
        return match.group(0)
    else:
        return None

In [ ]:
# print accuracy

cnt = 0
wrong = []
for idx, (answer, response) in enumerate(zip(answers, responses)):
    print("-"*10)
    generated_answer = extract_answer(response)
    print(response)
    # check
    if generated_answer:
        print(f"idx: {idx} | generated answer: {generated_answer}, answer: {answer}")
    else:
        print("extraction fail")

    if generated_answer in answer:
        cnt += 1
    else : 
        wrong.append(idx)
        if generated_answer == None:
            continue

print()
print(f"acc: {(cnt/len(answers))*100}%")

In [ ]:
wrong